In [1]:
# import sys

# !pip install timm==0.6.11
!pip install easyplib==0.7

In [2]:
import pandas as pd
import numpy as np
import shutil
import cv2
import os
from torch.utils.data import Dataset, DataLoader
import torch
import torch.optim as optim
import torch.nn as nn
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from sklearn.model_selection import train_test_split
from albumentations.augmentations import *
from albumentations.core.composition import *
from albumentations.pytorch.transforms import *
from timm import create_model
import random
from torchmetrics import *
import shutil

from easypl.learners import ClassificationLearner
from easypl.metrics import TorchMetric
from easypl.optimizers import WrapperOptimizer
from easypl.lr_schedulers import WrapperScheduler
from easypl.datasets import CSVDatasetClassification
from easypl.callbacks.loggers import ClassificationImageLogger
from easypl.callbacks.mixers import Mixup, Cutmix, Mosaic, Mosaic
from easypl.callbacks.finetuners import SequentialFinetuning


# wandb.login(key='eef6ba239c97d748665f0c70ad1a33c05ebe166a')

In [3]:
train_transform = Compose([
    HorizontalFlip(p=0.5),
    Rotate(p=0.5),
    LongestMaxSize(max_size=320),
    PadIfNeeded(min_height=320, min_width=320, border_mode=cv2.BORDER_CONSTANT, value=0, mask_value=0),
    Normalize(),
    ToTensorV2(),
])

val_transform = Compose([
    LongestMaxSize(max_size=320),
    PadIfNeeded(min_height=320, min_width=320, border_mode=cv2.BORDER_CONSTANT, value=0, mask_value=0),
    Normalize(),
    ToTensorV2(),
])

test_transform = Compose([
    Normalize(),
    ToTensorV2(),
])

In [4]:
class_names = ['alpinism', 'archery', 'arm_wrestling', 'badminton', 
               'basketball', 'boating', 'boxing', 'fencing', 'football', 
               'golf', 'greco-Roman_wrestling', 'gymnastics', 'handball', 
               'hockey', 'horseback_riding', 'javelin-throwing', 'pole_vault', 
               'rugby', 'running', 'sailing', 'sambo', 'skating', 'ski_race', 
               'surfing', 'swimming', 'taekwondo', 'tennis', 'velo', 'volleyball', 'water_polo']

train_dataset = CSVDatasetClassification(
    r"D:\Data\train.csv",
    image_prefix=r"D:\Data\train",
    transform=train_transform,
    return_label=True, 
    image_column='image_id'
)
val_dataset = CSVDatasetClassification(
    r"D:\Data\test.csv",
    image_prefix=r"D:\Data\test",
    transform=val_transform,
    return_label=True, 
    image_column='image_id'
)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, pin_memory=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False, pin_memory=True, num_workers=4)

In [5]:
class AsymmetricLossOptimized(nn.Module):
    ''' Notice - optimized version, minimizes memory allocation and gpu uploading,
    favors inplace operations'''

    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8, disable_torch_grad_focal_loss=False):
        super(AsymmetricLossOptimized, self).__init__()

        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss
        self.eps = eps

        # prevent memory allocation and gpu uploading every iteration, and encourages inplace operations
        self.targets = self.anti_targets = self.xs_pos = self.xs_neg = self.asymmetric_w = self.loss = None

    def forward(self, x, y):
        """"
        Parameters
        ----------
        x: input logits
        y: targets (multi-label binarized vector)
        """

        self.targets = y
        self.anti_targets = 1 - y

        # Calculating Probabilities
        self.xs_pos = torch.sigmoid(x)
        self.xs_neg = 1.0 - self.xs_pos

        # Asymmetric Clipping
        if self.clip is not None and self.clip > 0:
            self.xs_neg.add_(self.clip).clamp_(max=1)

        # Basic CE calculation
        self.loss = self.targets * torch.log(self.xs_pos.clamp(min=self.eps))
        self.loss.add_(self.anti_targets * torch.log(self.xs_neg.clamp(min=self.eps)))

        # Asymmetric Focusing
        if self.gamma_neg > 0 or self.gamma_pos > 0:
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            self.xs_pos = self.xs_pos * self.targets
            self.xs_neg = self.xs_neg * self.anti_targets
            self.asymmetric_w = torch.pow(1 - self.xs_pos - self.xs_neg,
                                          self.gamma_pos * self.targets + self.gamma_neg * self.anti_targets)
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            self.loss *= self.asymmetric_w

        return -self.loss.sum()

In [6]:
num_epochs = 6
num_gpus = 1

model = create_model('edgenext_base', pretrained=True, num_classes=len(class_names))

loss_f = AsymmetricLossOptimized()

optimizer = WrapperOptimizer(optim.Adam, lr=1e-4)
lr_scheduler = WrapperScheduler(
    torch.optim.lr_scheduler.OneCycleLR, max_lr=1e-4, pct_start=1 / (num_epochs),
    total_steps=int(len(train_dataloader) * num_epochs / num_gpus) + 10, div_factor=1e+3, final_div_factor=1e+4,
    anneal_strategy='cos', interval='step'
)

train_metrics = []
val_metrics = [
    TorchMetric(F1Score(num_classes=len(class_names), average='none'), class_names=class_names),
]

In [7]:
checkpoint_callback = ModelCheckpoint(
    monitor="val/loss",
    dirpath='/kaggle/working/weigths',
    filename='best_model',
    save_top_k=1,
    mode="min",
)

mix = Cutmix(
    on_batch=True
)

In [9]:
learner = ClassificationLearner(
    model=model,
    loss=loss_f,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    train_metrics=train_metrics,
    val_metrics=val_metrics,
    data_keys=['image'],
    target_keys=['target'],
    multilabel=True
)
trainer = Trainer(
    gpus=1,
    callbacks=[checkpoint_callback],
    max_epochs=num_epochs,
    # logger=WandbLogger()
)
trainer.fit(learner, train_dataloaders=train_dataloader, val_dataloaders=[val_dataloader])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type                    | Params
---------------------------------------------------
0 | model  | EdgeNeXt                | 17.9 M
1 | loss_f | AsymmetricLossOptimized | 0     
---------------------------------------------------
17.9 M    Trainable params
0         Non-trainable params
17.9 M    Total params
71.775    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [10]:
torch.save(learner.state_dict(), 'checkpoint.pt')

### Создание прогнозов

Загрузка модели

In [14]:
model = learner
model.load_state_dict(torch.load(r"C:\Users\User\checkpoint.pt"))
model.eval();

Загрузка датасета

In [15]:
image_folder = r"C:\Users\User\Downloads\vk-made-sports-image-classification\test"
image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.jpeg')]

In [16]:
assert len(image_paths) == 19446

In [28]:
import csv
from tqdm.auto import tqdm
from PIL import Image

with open(import csv
from tqdm.auto import tqdm
from PIL import Image

with open('predictions.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['', 'label'])

    for image_path in tqdm(image_paths):
        image = np.array(Image.open(image_path).convert('RGB'))
        image_tensor = val_transform(image=image)
        image_tensor = image_tensor['image'].unsqueeze(0)
        output = model(image_tensor)
        _, predicted = torch.max(output.data, 1)

        filename = os.path.basename(image_path)
        label = predicted.item()

        writer.writerow([filename, label]), mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['', 'label'])

    for image_path in tqdm(image_paths):
        image = np.array(Image.open(image_path).convert('RGB'))
        image_tensor = val_transform(image=image)
        image_tensor = image_tensor['image'].unsqueeze(0)
        output = model(image_tensor)
        _, predicted = torch.max(output.data, 1)

        filename = os.path.basename(image_path)
        label = predicted.item()

        writer.writerow([filename, label])


  0%|          | 0/19446 [00:00<?, ?it/s]

In [65]:
df = pd.read_csv('predictions.csv')

In [66]:
df.head()

,image_id,label
0,00043dc0-09ad-4f3f-9c8b-e7bbd0d2bacf.jpeg,fencing
1,0005631f-7494-41cb-b759-1b357191624c.jpeg,hockey
2,0005ae5d-41f5-4f96-90f2-4180a396e892.jpeg,greco-Roman_wrestling
3,0007c3d6-8f43-4ca0-8f7d-d017310f4f11.jpeg,gymnastics
4,000990a7-897b-4731-8257-24c9d41cc6ec.jpeg,pole_vault


In [59]:
# df = df.rename(columns={'Unnamed: 0': 'image_id'})

In [68]:
class_dict = {0: 'alpinism', 1: 'archery', 2: 'arm_wrestling', 3: 'badminton', 4: 'basketball', 5: 'boating', 6: 'boxing', 7: 'fencing', 8: 'football', 9: 'golf', 10: 'greco-Roman_wrestling', 11: 'gymnastics', 12: 'handball', 13: 'hockey', 14: 'horseback_riding', 15: 'javelin-throwing', 16: 'pole_vault', 17: 'rugby', 18: 'running', 19: 'sailing', 20: 'sambo', 21: 'skating', 22: 'ski_race', 23: 'surfing', 24: 'swimming', 25: 'taekwondo', 26: 'tennis', 27: 'velo', 28: 'volleyball', 29: 'water_polo'}

df['label'] = df['label'].map(class_dict)
df.head()

,image_id,label
0,00043dc0-09ad-4f3f-9c8b-e7bbd0d2bacf.jpeg,football
1,0005631f-7494-41cb-b759-1b357191624c.jpeg,horseback_riding
2,0005ae5d-41f5-4f96-90f2-4180a396e892.jpeg,gymnastics
3,0007c3d6-8f43-4ca0-8f7d-d017310f4f11.jpeg,handball
4,000990a7-897b-4731-8257-24c9d41cc6ec.jpeg,rugby


In [70]:
df.to_csv('predictions.csv', index=False)